# Bachelor Thesis 

### The Aim of this project is to create a software that will help forecasting swiss equity market,  help financials understand the companies situation and the market tendancies/behaviour in order to place trades.
### This software also will be able to place automatically weekly trades.

> > ### This jupyter notebook is for demonstration purposes only,  the code here will work perfectly however most of the good code practices will be violated,  the real software will be in separate files joined with this jupyter notebook in seperate folders

### For modularity purposes all the imports will stay in one cell 

In [27]:
import quandl
import os
import requests
import pickle
import time
import pandas as pd
import pprint
import math
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np
import alpaca_trade_api as tradeapi
from datetime import date, datetime
from IPython.display import display
from ipywidgets import widgets
from matplotlib import interactive
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from requests.exceptions import HTTPError
from alpaca_trade_api import rest
from alpaca_trade_api.rest import APIError
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
%matplotlib tk



## In this cell all the Constants will be gathered,  for software engineering purposes and to decouple some parts of the code this constants will be in a seperate file.
  > ### The base url is the most fundamental part of the constants. it's easier to call api's having
  > ### trying to find a website that provides financial data for free was a hard,  so special thank to alphavantage to provide very valuable financial data
   > ### The available_stocks_list variable will hold 16 stocks of the swiss equity market that accept to publish freely their **Cash Flow**,  **Balance sheet** and other detail about their **financial status**.
   > ### Those same 16 stocks will be the only ones traded


In [28]:
BASE_URL = """https://www.alphavantage.co/query?apikey=YCVISHALBF20DTQP"""
END_DATE = date.today()
SNB_FILENAME = "retrieve_update/src/data/indices/snb_data.pickle"
available_stocks_list = pickle.load(\
                                  open("retrieve_update/src/data/stocks/traded_stocks.pickle", "rb"))

### This is an optimisation function,  unfortunatuely there is no predefined function to append only and just only the lacking bits,  this will avoid everytime rewriting the datafiles,  since writing to disk is very expensive in term of ressources. also veryfing the file is up to date compared to the new data is very interesting to avoid useless updates. However comparing data which is melted with meta-data,  is very delicate and may turn to be very difficult. <br />  <br />  <br /> 



In [29]:
def is_updated(path_name, byte_data):
   pass

# This part is one of the backbones of the project and the critical part of EPIC2,  the main idea of this part is to use multiple plateforms to retrieve valubale informations
>> ### Each method used here will update weekly information of stocks,  indices etc... 
>> ### Full documentation of what is updated here will be found in the report
>> ### The Pickle objects format will be dictionnaries where the Keys are tickers/symbols ( IBM for example stands for IBM tech company) and the values are JSON objects/dictionnary objects containing informations related to those equities
>> ### The format will be also a pickle files. Actually serialized objects are more handy to use,  transport and extract
>> ### After a benchmark done with tinydb and other technologies,  Pickle was the most lightweight Technology to use which I believe that match my usecase I don't have terabytes of data to manipulate.

## as a first step we are going to update indices for equities 

In [30]:
def update_EMA():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        ### to respect maximum 5 API calls per minute
        if(nb_api_call  == 5 ):
            time.sleep(60)
            nb_api_call = 0 
           
        data = requests.get(BASE_URL+"&interval=weekly&time_period=15&series_type=close&symbol={}&function=EMA"
                      .format(ticker))
        
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        
        file = open("retrieve_update/src/data/indices/ema_index.pickle",  "wb" )
        pickle.dump(dataset, file)
        file.close()

def update_stoch_index():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
         ### to respect maximum 5 API calls per minute
        if(nb_api_call  == 5 ):
            time.sleep(60)
            nb_api_call = 0 
            #make a get request to the API 
        data = requests.get(BASE_URL+"&interval=weekly&symbol={}&function=stoch"
                      .format(ticker))
        
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        print(data.json())
        file = open( "retrieve_update/src/data/indices/stochastic_index.pickle",  "wb" )
        pickle.dump(dataset, file)
        file.close()

def update_MACD():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0 
        data = requests.get(BASE_URL+"&interval=weekly&series_type=close&symbol={}&function=macd"
                      .format(ticker))
        
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        file = open( "retrieve_update/src/data/indices/macd_index.pickle",  "wb" )
        pickle.dump(dataset, file)
        file.close()
    
def update_RSI():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0 
        data = requests.get(BASE_URL+"&time_period=10&interval=weekly&series_type=close&symbol={}&function=rsi"
                      .format(ticker))
        
        nb_api_call = nb_api_call+1; 
        print(data.json())
        dataset[ticker] = data.json()
        file = open( "retrieve_update/src/data/indices/rsi_index.pickle",  "wb" )
        pickle.dump(dataset, file)
        file.close()  

def update_CCI():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0 
        data = requests.get(BASE_URL+"&time_period=10&interval=weekly&series_type=close&symbol={}&function=cci"
                      .format(ticker))
        
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        print(data.json())
        file = open( "retrieve_update/src/data/indices/cci_index.pickle",  "wb" )
        pickle.dump(dataset, file)
        file.close()

def update_aroon():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0 
        data = requests.get(BASE_URL+"&time_period=10&interval=weekly&symbol={}&function=aroon"
                      .format(ticker))
        
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        #Write to file in pickle mode 
        file = open( "retrieve_update/src/data/indices/aroon_index.pickle",  "wb" )
        pickle.dump(dataset, file)
        file.close()  


## The second part will consist of retrieving some more financial insights about the companies Example: Balance sheet,  Cash flow etc... 
## Also we will fetch historical stocks prices for the traded equities

In [31]:
def update_SNB():
    snb_data = quandl.get("SNB/CAPCHSTOCKI",  authtoken = "kVxxtbafwx1jrcH7xwps", 
               collapse = "weekly", end_date = END_DATE)
    snb_data = snb_data.dropna()
    file = open(SNB_FILENAME,  "wb")
    pickle.dump(snb_data, file)
    file.close()

def update_stock_prices():
    #prepare json object to contain ticker : Weekly stock prices of ticker
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call == 5):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0
        try:
            data = requests.get(BASE_URL+"&symbol={}&function=TIME_SERIES_WEEKLY"
                      .format(ticker))
        except requests.exceptions.Timeout:
            print("Time out Exceeded please verify network connection or URL validity")
        except requests.exceptions.RequestException as e:  
            raise SystemExit(e)
        nb_api_call = nb_api_call+1;
        ### In case of sucess the weekly time Series Json element contains all stocks prices
        if("Weekly Time Series" in data.json() ):        
            dataset[ticker] = data.json()  
    ## this will apply to all update_index methods also
    #open file in write bytes modes
    file = open( "retrieve_update/src/data/stocks/stock_prices.pickle",  "wb" )
    #dump dataset 
    pickle.dump(dataset, file)
    file.close()
           
def update_overview():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call == 5):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0
        data = requests.get(BASE_URL+"&symbol={}&function=OVERVIEW"
                      .format(ticker))
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        print(data.json())
    file = open( "retrieve_update/src/data/stocks/stock_overview.pickle",  "wb" )
    pickle.dump(dataset, file)
    file.close()

def update_income_state():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0
        data = requests.get(BASE_URL+"&symbol={}&function=INCOME_STATEMENT"
                      .format(ticker))
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        print(data.json())
    file = open( "retrieve_update/src/data/stocks/income_statments.pickle",  "wb" )
    pickle.dump(dataset, file)
    file.close()
    
    
def update_balance_sheet():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0
        data = requests.get(BASE_URL+"&symbol={}&function=BALANCE_SHEET"
                      .format(ticker))
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        print(data.json())
    file = open( "retrieve_update/src/data/stocks/balance_sheet.pickle",  "wb" )
    pickle.dump(dataset, file)
    file.close()
    
def update_cash_flow():
    dataset = {}
    nb_api_call = 0
    for ticker in available_stocks_list:    
        if(nb_api_call  == 5 ):
            ### to respect maximum 5 API calls per minute
            time.sleep(60)
            nb_api_call = 0
        data = requests.get(BASE_URL+"&symbol={}&function=CASH_FLOW"
                      .format(ticker))
        nb_api_call = nb_api_call+1;    
        dataset[ticker] = data.json()
        
    file = open( "retrieve_update/src/data/stocks/cash_flow.pickle",  "wb" )
    pickle.dump(dataset, file)
    file.close()
def update_all():
    update_SNB()
    update_balance_sheet()
    time.sleep(60)
    update_cash_flow()
    time.sleep(60)
    update_stock_prices()
    time.sleep(60)
    update_aroon()
    time.sleep(60)
    update_CCI()
    time.sleep(60)
    update_EMA()
    time.sleep(60)
    update_MACD()
    time.sleep(60)
    update_RSI()
    time.sleep(60)
    update_stoch_index()
#update_all()

# This part of the project is the backbone of Epic3
>> ## This part will take raw data ( gerenally JSON ) and transforme it into a more manipulable format in order to extract information and help a developer create his dashboard in a fluid way
>> ## As a first step I'll make sure to modify the JSON objects ( stored in dictionnaries) and transform them into pandas dataframes,  this is the most "usable" format and the most documented one
>> ## The next step will consist of validating all fields/rows etc.
>> ## furthermore,  more preprocessing will be done to ensure that values are stored as float64 and date values are normalized
>> ## The code here will automatically persist the data in a folder AKA data warehouse  



In [32]:
# In this cell some helper functions and global variables are going to be declared
class CorruptedFields(Exception):
    pass



def clean_helper(df: pd.DataFrame):
    #Calculate sum of NaN values
    fields = df.isna().sum()
    if sum(fields) !=  0:
        raise CorruptedFields


## The first part of Epic 3 will consist on creating truth tables for Machine learning Training 
>> ### The indices / prices / traded volume will be our feautres in the training
>> ### The Labels will be as follow 'Up trend' 'Down trend' respectively 1 and -1 in Dataframes.

In [33]:
#list of dictionnaries mapped this way symbol : data
stepone_rsi = {symbol:None for symbol in available_stocks_list}
stepone_cci = {symbol:None for symbol in available_stocks_list}
stepone_aroon = {symbol:None for symbol in available_stocks_list}
stepone_ema = {symbol:None for symbol in available_stocks_list}
stepone_macd = {symbol:None for symbol in available_stocks_list}
stepone_stoch = {symbol:None for symbol in available_stocks_list}
# a list of all indices dictionanires in order to automate certain jobs for instance joining dataframes by symbol
clean_indices = [stepone_aroon, stepone_cci, stepone_ema, stepone_macd, stepone_rsi, stepone_stoch]
# an empty dictionnary 
joined_symbols = {symbol:None for symbol in available_stocks_list}

stock_prices = {symbol:None for symbol in available_stocks_list}
# This function will take stocks transform them from json to df,  
# store them,  and then save them in CSV format
def preprocess_stocks_prices():
    # Read raw data
    raw_data = pickle.load(open("retrieve_update/src/data/stocks/stock_prices.pickle", "rb"))
    for symbol in raw_data:    
        # removing Meta-data 
        (raw_data[symbol]).pop('Meta Data',  None)
        # get the JSON object containing the prices
        x, raw_data[symbol] = raw_data[symbol].popitem()
        for date in raw_data[symbol]:
                for value in raw_data[symbol][date]:
                    #transform strings into float values
                    raw_data[symbol][date][value] = float(raw_data[symbol][date][value])
        #Create Dataframe from dictionnary and replace the JSON object by dataframe in raw data Json Object
        raw_data[symbol] = (pd.DataFrame.from_dict(raw_data[symbol])).transpose()
        # The most important "feature" in our data is prices 
        # if corrupted prices are found this may affect the process of plotting or model trainin
        # In order to avoid any prolems an exception will be trigered 
        try:
            clean_helper(raw_data[symbol])
        except CorruptedFields:
            print("""Corrupted Fields detected please relaunch the program 
            or contact the Admin error occured on {} """.format(symbol))
        #keep stock_prices dictionnary containing symbol -> dataframe representing symbols for later-use
        stock_prices[symbol] = raw_data[symbol]
        raw_data[symbol].to_csv("clean_preprocess/data_warehouse/stocks/{}_clean.csv".format(symbol))
        #isplay(raw_data[symbol])


        
def preprocess_rsi():
    data = pickle.load(open("retrieve_update/src/data/indices/rsi_index.pickle", "rb"))
    for symbol in data:
        print(data[symbol])
        (data[symbol]).pop('Meta Data',  None)
        x, data[symbol] = data[symbol].popitem()
        for obj in data[symbol]:
            for value in data[symbol][obj]:
                data[symbol][obj][value] = float( data[symbol][obj][value]) 
        stepone_rsi[symbol] = pd.DataFrame.from_dict(data[symbol]).transpose()
        #normalize date for all dataframes
        stepone_rsi[symbol].index = [x[0:10] for x in stepone_rsi[symbol].index.values.tolist()]
        #display(stepone_ema[symbol])
        
def preprocess_cci():
    data = pickle.load(open("retrieve_update/src/data/indices/cci_index.pickle", "rb"))
    for symbol in data:  
        (data[symbol]).pop('Meta Data',  None)
        x, data[symbol] = data[symbol].popitem()
        for obj in data[symbol]:
            for value in data[symbol][obj]:
                data[symbol][obj][value] = float( data[symbol][obj][value])            
        stepone_cci[symbol] = pd.DataFrame.from_dict(data[symbol]).transpose()
        #normalize date for all dataframes
        stepone_cci[symbol].index = [x[0:10] for x in stepone_cci[symbol].index.values.tolist()]
        #display(stepone_cci[symbol])

    
def preprocess_aroon():
    data = pickle.load(open("retrieve_update/src/data/indices/aroon_index.pickle", "rb"))
    for symbol in data:  
        (data[symbol]).pop('Meta Data',  None)
        x, data[symbol] = data[symbol].popitem()
        for obj in data[symbol]:
            for value in data[symbol][obj]:
                data[symbol][obj][value] = float( data[symbol][obj][value])            
        stepone_aroon[symbol] = pd.DataFrame.from_dict(data[symbol]).transpose()
        #normalize date for all dataframes
        stepone_aroon[symbol].index = [x[0:10] for x in stepone_aroon[symbol].index.values.tolist()]
        #display(stepone_aroon[symbol])
    
    
def preprocess_ema():
    data = pickle.load(open("retrieve_update/src/data/indices/ema_index.pickle", "rb"))
    for symbol in data:  
        (data[symbol]).pop('Meta Data',  None)
        x, data[symbol] = data[symbol].popitem()
        for obj in data[symbol]:
            for value in data[symbol][obj]:
                data[symbol][obj][value] = float( data[symbol][obj][value])            
        stepone_ema[symbol] = pd.DataFrame.from_dict(data[symbol]).transpose()
        #normalize date for all dataframes
        stepone_ema[symbol].index = [x[0:10] for x in stepone_ema[symbol].index.values.tolist()]
        #display(stepone_ema[symbol])
def preprocess_macd():
    data = pickle.load(open("retrieve_update/src/data/indices/macd_index.pickle", "rb"))
    for symbol in data:  
        (data[symbol]).pop('Meta Data',  None)
        x, data[symbol] = data[symbol].popitem()
        for obj in data[symbol]:
            for value in data[symbol][obj]:
                data[symbol][obj][value] = float( data[symbol][obj][value])            
        stepone_macd[symbol] = pd.DataFrame.from_dict(data[symbol]).transpose()
        #normalize date for all dataframes
        stepone_macd[symbol].index = [x[0:10] for x in stepone_macd[symbol].index.values.tolist()]
        #display(stepone_macd[symbol])
        
def preprocess_stoch():
    data = pickle.load(open("retrieve_update/src/data/indices/stochastic_index.pickle", "rb"))
    for symbol in data:  
        print(data[symbol])
        (data[symbol]).pop('Meta Data',  None)
        x, data[symbol] = data[symbol].popitem()
        for obj in data[symbol]:
            for value in data[symbol][obj]:
                data[symbol][obj][value] = float( data[symbol][obj][value])            
        stepone_stoch[symbol] = pd.DataFrame.from_dict(data[symbol]).transpose()
        #normalize date for all dataframes
        stepone_stoch[symbol].index = [x[0:10] for x in stepone_stoch[symbol].index.values.tolist()]
        #display(stepone_stoch[symbol])

    
def stepone_preprocess_all():
    preprocess_stocks_prices()
    preprocess_rsi()
    preprocess_cci()
    preprocess_aroon()
    preprocess_ema()
    preprocess_macd()
    preprocess_stoch()
    
def normalize_date():
    pass

def join_data():
    for index in clean_indices:
        for symbol in index:
            if(joined_symbols[symbol] is None):
                joined_symbols[symbol] = index[symbol]
            else:
                joined_symbols[symbol] = pd.concat([index[symbol], joined_symbols[symbol]], axis = 1)
def join_indices_and_prices():
    for symbol in stock_prices:
        #left join indices and stocks prices
        joined_symbols[symbol] = (joined_symbols[symbol]).\
        merge(stock_prices[symbol], left_index = True, right_index = True, how = 'left')

        

    
                
stepone_preprocess_all()
join_data()
join_indices_and_prices()

{'Meta Data': {'1: Symbol': 'ALC', '2: Indicator': 'Relative Strength Index (RSI)', '3: Last Refreshed': '2021-06-14', '4: Interval': 'weekly', '5: Time Period': 10, '6: Series Type': 'close', '7: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: RSI': {'2021-06-14': {'RSI': '52.6899'}, '2021-06-11': {'RSI': '51.2902'}, '2021-06-04': {'RSI': '50.4571'}, '2021-05-28': {'RSI': '48.7136'}, '2021-05-21': {'RSI': '45.5577'}, '2021-05-14': {'RSI': '44.4152'}, '2021-05-07': {'RSI': '49.0753'}, '2021-04-30': {'RSI': '67.1471'}, '2021-04-23': {'RSI': '68.7795'}, '2021-04-16': {'RSI': '65.4716'}, '2021-04-09': {'RSI': '64.5536'}, '2021-04-01': {'RSI': '58.7682'}, '2021-03-26': {'RSI': '55.1414'}, '2021-03-19': {'RSI': '50.7783'}, '2021-03-12': {'RSI': '51.9803'}, '2021-03-05': {'RSI': '50.3951'}, '2021-02-26': {'RSI': '51.5126'}, '2021-02-19': {'RSI': '60.7965'}, '2021-02-12': {'RSI': '67.5858'}, '2021-02-05': {'RSI': '72.8234'}, '2021-01-29': {'RSI': '66.9947'}, '2021-01-22': {'RSI': '75.561

{'Meta Data': {'1: Symbol': 'SENS', '2: Indicator': 'Relative Strength Index (RSI)', '3: Last Refreshed': '2021-06-14', '4: Interval': 'weekly', '5: Time Period': 10, '6: Series Type': 'close', '7: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: RSI': {'2021-06-14': {'RSI': '71.1539'}, '2021-06-11': {'RSI': '72.1332'}, '2021-06-04': {'RSI': '61.6946'}, '2021-05-28': {'RSI': '48.2005'}, '2021-05-21': {'RSI': '47.0146'}, '2021-05-14': {'RSI': '45.5180'}, '2021-05-07': {'RSI': '44.9936'}, '2021-04-30': {'RSI': '48.1943'}, '2021-04-23': {'RSI': '46.4040'}, '2021-04-16': {'RSI': '44.8311'}, '2021-04-09': {'RSI': '50.2620'}, '2021-04-01': {'RSI': '53.3650'}, '2021-03-26': {'RSI': '55.5513'}, '2021-03-19': {'RSI': '62.7668'}, '2021-03-12': {'RSI': '61.8410'}, '2021-03-05': {'RSI': '55.6096'}, '2021-02-26': {'RSI': '69.5311'}, '2021-02-19': {'RSI': '97.9348'}, '2021-02-12': {'RSI': '97.5908'}, '2021-02-05': {'RSI': '96.2327'}, '2021-01-29': {'RSI': '94.6463'}, '2021-01-22': {'RSI': '94.45

{'Meta Data': {'1: Symbol': 'ADXN', '2: Indicator': 'Relative Strength Index (RSI)', '3: Last Refreshed': '2021-06-14', '4: Interval': 'weekly', '5: Time Period': 10, '6: Series Type': 'close', '7: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: RSI': {'2021-06-14': {'RSI': '51.3680'}, '2021-06-11': {'RSI': '54.6569'}, '2021-06-04': {'RSI': '46.0159'}, '2021-05-28': {'RSI': '40.7307'}, '2021-05-21': {'RSI': '44.6664'}, '2021-05-14': {'RSI': '41.9476'}, '2021-05-07': {'RSI': '43.3158'}, '2021-04-30': {'RSI': '42.5056'}, '2021-04-23': {'RSI': '46.6565'}, '2021-04-16': {'RSI': '44.5425'}, '2021-04-09': {'RSI': '43.7624'}, '2021-04-01': {'RSI': '45.6187'}, '2021-03-26': {'RSI': '45.3404'}, '2021-03-19': {'RSI': '47.1496'}, '2021-03-12': {'RSI': '46.3403'}, '2021-03-05': {'RSI': '45.4451'}, '2021-02-26': {'RSI': '45.7474'}, '2021-02-19': {'RSI': '50.2292'}, '2021-02-12': {'RSI': '48.0798'}, '2021-02-05': {'RSI': '47.7899'}, '2021-01-29': {'RSI': '46.1795'}, '2021-01-22': {'RSI': '49.01

{'Meta Data': {'1: Symbol': 'ALC', '2: Indicator': 'Stochastic (STOCH)', '3: Last Refreshed': '2021-06-14', '4: Interval': 'weekly', '5.1: FastK Period': 5, '5.2: SlowK Period': 3, '5.3: SlowK MA Type': 0, '5.4: SlowD Period': 3, '5.5: SlowD MA Type': 0, '6: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: STOCH': {'2021-06-14': {'SlowD': '54.2448', 'SlowK': '70.7692'}, '2021-06-11': {'SlowD': '40.8651', 'SlowK': '55.3225'}, '2021-06-04': {'SlowD': '30.7061', 'SlowK': '36.6426'}, '2021-05-28': {'SlowD': '33.0431', 'SlowK': '30.6302'}, '2021-05-21': {'SlowD': '45.3821', 'SlowK': '24.8455'}, '2021-05-14': {'SlowD': '68.2872', 'SlowK': '43.6537'}, '2021-05-07': {'SlowD': '86.5700', 'SlowK': '67.6471'}, '2021-04-30': {'SlowD': '96.6560', 'SlowK': '93.5607'}, '2021-04-23': {'SlowD': '93.8187', 'SlowK': '98.5023'}, '2021-04-16': {'SlowD': '80.8344', 'SlowK': '97.9050'}, '2021-04-09': {'SlowD': '60.2122', 'SlowK': '85.0488'}, '2021-04-01': {'SlowD': '39.2898', 'SlowK': '59.5494'}, '2021-0

{'Meta Data': {'1: Symbol': 'PLAN', '2: Indicator': 'Stochastic (STOCH)', '3: Last Refreshed': '2021-06-14', '4: Interval': 'weekly', '5.1: FastK Period': 5, '5.2: SlowK Period': 3, '5.3: SlowK MA Type': 0, '5.4: SlowD Period': 3, '5.5: SlowD MA Type': 0, '6: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: STOCH': {'2021-06-14': {'SlowK': '43.1092', 'SlowD': '37.0416'}, '2021-06-11': {'SlowK': '36.1552', 'SlowD': '32.6262'}, '2021-06-04': {'SlowK': '31.8604', 'SlowD': '28.8751'}, '2021-05-28': {'SlowK': '29.8628', 'SlowD': '29.2727'}, '2021-05-21': {'SlowK': '24.9021', 'SlowD': '37.6431'}, '2021-05-14': {'SlowK': '33.0531', 'SlowD': '54.1187'}, '2021-05-07': {'SlowK': '54.9741', 'SlowD': '69.9308'}, '2021-04-30': {'SlowK': '74.3288', 'SlowD': '72.4518'}, '2021-04-23': {'SlowK': '80.4894', 'SlowD': '61.0725'}, '2021-04-16': {'SlowK': '62.5370', 'SlowD': '38.5967'}, '2021-04-09': {'SlowK': '40.1912', 'SlowD': '20.7914'}, '2021-04-01': {'SlowK': '13.0618', 'SlowD': '11.1025'}, '2021-

{'Meta Data': {'1: Symbol': 'AAA', '2: Indicator': 'Stochastic (STOCH)', '3: Last Refreshed': '2021-06-14', '4: Interval': 'weekly', '5.1: FastK Period': 5, '5.2: SlowK Period': 3, '5.3: SlowK MA Type': 0, '5.4: SlowD Period': 3, '5.5: SlowD MA Type': 0, '6: Time Zone': 'US/Eastern Time'}, 'Technical Analysis: STOCH': {'2021-06-14': {'SlowK': '57.6295', 'SlowD': '41.7685'}, '2021-06-11': {'SlowK': '37.7050', 'SlowD': '30.0633'}, '2021-06-04': {'SlowK': '29.9711', 'SlowD': '23.0972'}, '2021-05-28': {'SlowK': '22.5137', 'SlowD': '18.6515'}, '2021-05-21': {'SlowK': '16.8066', 'SlowD': '20.2931'}, '2021-05-14': {'SlowK': '16.6341', 'SlowD': '28.7690'}, '2021-05-07': {'SlowK': '27.4386', 'SlowD': '40.5214'}, '2021-04-30': {'SlowK': '42.2342', 'SlowD': '47.7114'}, '2021-04-23': {'SlowK': '51.8914', 'SlowD': '46.9408'}, '2021-04-16': {'SlowK': '49.0087', 'SlowD': '44.4950'}, '2021-04-09': {'SlowK': '39.9223', 'SlowD': '43.5689'}, '2021-04-01': {'SlowK': '44.5539', 'SlowD': '47.3789'}, '2021-0

## As a second part of preprocessing we will need to deal with NaN values and make a cleaning strategy
>> ### The first intuition is to drop columns with NaN values
>> ### otherwise we could fill them following a strategy using the astonishing improvement made by Mr MeeseeksMachine and released on github under this source https://github.com/pandas-dev/pandas/blob/v1.2.4/pandas/core/frame.py#L4452-L4469,  the contribution added fillna() method which helps filling the gap with multiple intelligent strategies,  I would have prefered a method to fill it with the average/ median between 1 to n previous and next cells. I may consider making this improvement myself and make a pull request
>> ### However for simplicity and the good amount of data we have we will just drop columns with NaN values.

In [34]:
def data_cleaning():
    for symbol in joined_symbols:
        joined_symbols[symbol] = joined_symbols[symbol].dropna(axis=0)
        #joined_symbols[symbol] = joined_symbols[symbol].fillna(method = 'bfill', axis = 0)
data_cleaning()


## The final part is to label the data manually by adding a column "trend" and labeling the data by trend 
>> ### "Up"and "down" will refer to an augmentation/diminuation in the average price and "Neutral" will define a price still unchanged
>> ### It's important to say if the price changed significally we will set it as a price trend the threshhold will be 0.005 in the price movement
>> ### Due to warnings in pandas docs about iterating over Dataframes ( slowly and risky ) we will have to make a turnaround in order to be able to make a lable list 

### Before tstarting the labeling process I remarked that there is a stock of an almost banckrupted company which is ADBN : Americana Distribution 
>> #### It's porbably going to be delisted from the swiss stock exchange so I'm going to get it off the traded stocks 
>> #### If I have enough time i'll see if I can make sure to take off automatically banckrupted firms
### Also AAA is a new company,  it have listed its stocks on the market by the end of 2020 


>> #### Both of the stocks will be unfortunately deleted from the traded stocks which will leave us with 14 traded stocks

In [35]:
joined_symbols.pop('ADBN')
joined_symbols.pop('AAA')
available_stocks_list.remove('ADBN')
available_stocks_list.remove('AAA') 

## In this part we will prepare the data to be fed to the machine learning Algorithm so we need to label it into 3 classes Up,  down and neutral which will describe the average trend of the prices.  

In [36]:
def label_helper(x, y):
    margin = (y-x)/y
    if margin<0:
        return -1
    else:
        return 1
# To train a classification ML algorithm we need to have labels with this function 
# we will ensure labeling to 2 classes Up(Overbought) ,  down(Oversold) respectively 1, -1
# The labels are integers so ML algorithms could easily process them
def label_data():
    
    label_list = [1]
    for symbol in joined_symbols:
        
        df = pd.concat([joined_symbols[symbol]['1. open'], joined_symbols[symbol]['4. close']], axis = 1)
        df['avg'] = df.mean(axis = 1)
        avg_list = df['avg']
        for i in range(0, avg_list.size-1):
            label_list.append(label_helper(avg_list[i+1], avg_list[i]))
        joined_symbols[symbol] = pd.concat([joined_symbols[symbol], pd.Series(label_list, 
                                    index = joined_symbols[symbol].index, name = "Trend label")], axis = 1)
        label_list = [1]
        
        
label_data()        

### After having our data clean,  labeled and ready to be feeded to a Machine Learning Algorithm I'm going to persist under csv format a data warehouse 

In [37]:
# Persist Data in datawarehouse
for symbol in joined_symbols:
    #display(joined_symbols[symbol])
    joined_symbols[symbol].to_csv("clean_preprocess/data_warehouse/data_for_ML/{}_data.csv".format(symbol), index = False)   

## In order to help financials have an insight about the market another kind of data is useful,  being able to find highly correlated Pair(index,  stock) is actually a trading strategy 
>> ### In this part I'm going to use the Swiss National Bank Performance index for multiple fields and try to find a correlation with a stock I'm going to plot 
>> ### In Order to that I'm going to use a well known mathematical equation,  actually the best way to find correlation between two discrete variables is covariance,  correlation is the scaled version of covariance
>> ### Before that a long benchmark had to be done comparing the 3 famous correlation indicators 
>> ### Those 3 correlation indicator's are Pearson,  Kendall and Spearman ratios 
>> ### each one has a use case depending on some aspects of the data.
>> # **Since we can't make any assumptions about the data and it's hard to say that weekly data is linear we will use both spearman and pearman's correlation coefficient since financials are interested in data exploration rather than rigid truth**
>> # The equation is simple and stated below 
$$  \eta = \frac{cov(X, Y)}{\sigma(X)\times \sigma(Y)} $$
>> ### Cov means the statistical covariance and sigma referes to the standard deviation the same equation applies for both correlation ratios,  for Spearman the variables are the ranks however for pearman's the cariable X and Y are the actual values

In [38]:
# Load SNB data from our data lake
snb_data = pickle.load(open(SNB_FILENAME, "rb"))
#Transform it into a dict
snb_arrays = snb_data.to_dict(orient = 'list')
#generate a dictionnary to fill with key : company symbol ,  value : dataframe of company's open price
symbol_prices = {symbol:None for symbol in available_stocks_list}
# in order to find correlations we need to observe open prices in the market
def load_prices():
    for symbol in available_stocks_list:
        price_list = pd.read_csv("clean_preprocess/data_warehouse/stocks/{}_clean.csv".format(symbol))["1. open"]
        symbol_prices[symbol] = price_list
#find correlations indexes between an index and a company price and persist them
def find_correlations():
    #prepare dataframe to be filled
    spearman_corr = pd.DataFrame(None, index = [w for w in symbol_prices], columns = [w for w in snb_arrays])
    pearson_corr = pd.DataFrame(None, index = [w for w in symbol_prices], columns = [w for w in snb_arrays])
    for symbol in symbol_prices:
        for index in snb_arrays:
            if(len(snb_arrays[index])>len(symbol_prices[symbol])):
                #make sure that while finding correlations that we have the same data lenght
                sublist = snb_arrays[index][-len(symbol_prices[symbol]):]
                pearson_corr.at[(symbol, index)], y = scipy.stats.pearsonr(sublist, symbol_prices[symbol])
                spearman_corr.at[symbol, index], y = scipy.stats.spearmanr(sublist, symbol_prices[symbol])
            else:
                #make sure that while finding correlations that we have the same data lenght
                sublist = symbol_prices[symbol][-len(snb_arrays[index]):]
                pearson_corr.at[(symbol, index)], y = scipy.stats.pearsonr(sublist, snb_arrays[index])
                spearman_corr.at[symbol, index], y = scipy.stats.spearmanr(sublist, snb_arrays[index])
    display(spearman_corr)
    spearman_corr.to_csv("clean_preprocess/data_warehouse/data_for_dashboard/spearman_matrix.csv")
    pearson_corr.to_csv("clean_preprocess/data_warehouse/data_for_dashboard/pearson_matrix.csv")
load_prices()
find_correlations()

,SPI Swiss Performance Index - Index total (including dividend reinvestment),SPI Swiss Performance Index - Banks,SPI Swiss Performance Index - Financial services,SPI Swiss Performance Index - Insurance,SPI Swiss Performance Index - Food and beverages,SPI Swiss Performance Index - Health care,SPI Swiss Performance Index - Registered shares,SPI Swiss Performance Index - Bearer shares and participation certificates,SMI Swiss Market Index (excluding dividend reinvestment)
ALC,-0.371656,0.404965,-0.377618,0.08765,-0.482319,-0.515911,-0.290645,-0.605428,-0.286105
SIGN,0.555467,-0.11962,0.585978,0.557779,0.454539,0.555554,0.56845,0.50703,0.516897
CEVA,-0.790564,0.508604,-0.786169,-0.744825,-0.866162,-0.759802,-0.783335,-0.788974,-0.646175
MED,-0.819402,0.707097,-0.768758,-0.587419,-0.913723,-0.842734,-0.786932,-0.883176,-0.633746
SENS,0.47782,-0.214731,0.460455,0.532923,0.480841,0.513256,0.465884,0.498964,0.403825
LAND,-0.215287,-0.407354,-0.215735,-0.29146,-0.176826,-0.15465,-0.229922,-0.162492,-0.143153
GALE,0.692217,-0.058631,0.653643,0.673917,0.674163,0.751348,0.68258,0.72063,0.704177
PLAN,-0.681305,0.371494,-0.681392,-0.437128,-0.519969,-0.543174,-0.68488,-0.492867,-0.628441
BRG,0.609717,-0.149496,0.640115,0.572693,0.592954,0.597233,0.599755,0.595699,0.464747
RARE,-0.272681,0.124716,-0.23291,-0.162568,-0.253963,-0.34514,-0.259143,-0.280071,-0.239502


# As from Now we have all our data cleaned,  preprocessed and ready for use
>>## In this part we will give the user the ability to explore the companies balance sheets and cash flows insights 
>>## This part is restricted due to the low amount of Data that we have,  still this project will still be working if the data is updated or any other data sources are added ( for example preminium data from financial providers)

In [39]:
## The Next cell will contain helper methods to prepare the data to be plotted.
## For scalabilty purposes this part will be independent and will use raw data instead of Data warehouses

In [40]:
#a lambda function to clean the dataframe using applymap
value_or_nan = lambda  x : int(x)/1000000 if str(x).isnumeric() else pd.NA 
#transform from dict to df with cleaning process and building a dataframe containing balance sheet
def df_from_balance_sheet(data):
    #prepare index list which is a date list
    date_list = [w["fiscalDateEnding"] for w in data]
    #prepare columns names list 
    value_list = ["total assets", "total liabilities", "current assets"\
                , "current liabilities", "inventory", "goodwill"]
    #inzitialise the dataframe with colmuns and index list 
    df = pd.DataFrame(None, index = date_list, columns = value_list)
    #Fill the data frame
    for key in data:
        df.at[key["fiscalDateEnding"], "total assets"] = key["totalAssets"]
        df.at[key["fiscalDateEnding"], "total liabilities"] = key["totalLiabilities"]
        df.at[key["fiscalDateEnding"], "current assets"] = key["totalNonCurrentAssets"]
        df.at[key["fiscalDateEnding"], "current liabilities"] = key["totalCurrentLiabilities"]
        df.at[key["fiscalDateEnding"], "inventory"] = key["inventory"]
        df.at[key["fiscalDateEnding"], "goodwill"] = key["goodwill"]
    # clean and transform data to appropriate data type
    df = df.applymap(value_or_nan)
    # clean NaN values with threshhold 4
    df = df.dropna(axis = 'index', thresh = 4)
    df = df.dropna(axis = 'columns', thresh = 4)
    return df

def df_from_cash_flow(data):
    #prepare index list which is a date list
    date_list = [w["fiscalDateEnding"] for w in data]
    #prepare columns names list 
    value_list = ["profit loss", "net income", "dividend payout"]
    #inzitialise the dataframe with colmuns and index list 
    df = pd.DataFrame(None, index = date_list, columns = value_list)
    #Fill the data frame
    for key in data:
        df.at[key["fiscalDateEnding"], "profit/loss"] = key["netIncome"]
        df.at[key["fiscalDateEnding"], "net income"] = key["profitLoss"]
        df.at[key["fiscalDateEnding"], "dividend payout"] = key["dividendPayout"]
    # clean and transform data to appropriate data type
    df = df.applymap(value_or_nan)
    # clean NaN values with threshhold 4
    df = df.dropna(axis = 'index', thresh = 2)
    df = df.dropna(axis = 'columns', thresh = 3)
    return df


In [41]:
class CompanyFin:
    """
    this class will contain all the data necessary for the company to be plotted in graphs
    a class is requiered to keep track of each detail of the data,  dataframes don't work as good as classes
    in this case we don't have homogenous data and we want always to keep metadata of the company
    """
    # static attributes containing company's overview,  balance sheet and cashflow 
    # they are imported from the raw data
    # they are important to build company's class,  all companies share theses attributes 
    # we could see it as a shared database among class instances 
    overview = pickle.load(open("retrieve_update/src/data/stocks/stock_overview.pickle",  "rb"))
    balance_sheet = pickle.load(open( "retrieve_update/src/data/stocks/balance_sheet.pickle",  "rb" ))
    cash_flow = pickle.load(open("retrieve_update/src/data/stocks/cash_flow.pickle",  "rb" ))
    
    def __init__(self,  symbol):
        self.symbol = symbol
        if(bool(CompanyFin.balance_sheet[symbol])):
            simple_sheet = CompanyFin.balance_sheet[symbol]["annualReports"]
            if(not df_from_balance_sheet(simple_sheet).empty):
                 self.balance_sheet = df_from_balance_sheet(simple_sheet)
            else:
                 self.balance_sheet = None
                    
        else:
            self.balance_sheet = None
        
        if(bool(CompanyFin.cash_flow[symbol])):
            simple_flow = CompanyFin.cash_flow[symbol]["annualReports"]
            if(not df_from_cash_flow(simple_flow).empty):
                 self.cash_flow = df_from_cash_flow(simple_flow)
            else:
                self.cash_flow = None
        else:
            self.cash_flow = None
            


In [42]:
# A list holding company's objects filled with company's data
companylist = []
# function to create and append company's objects
def generate_cmp_list():
    for k in available_stocks_list:
        c = CompanyFin(k)
        companylist.append(c)

generate_cmp_list()

In [43]:
interactive(True)
%matplotlib tk

# a function to take a dataframe of a company and it's symbol and plot it using tkinter GUI
def display_by_company(df, symbol):
    df = df[::-1]
    display(df)
    #ase = plt.subplots_adjust(left = 0.08,  bottom = 0.18,  right = 0.9,  top = 0.88,  wspace = 0.4,  hspace = 0.68)
    df.plot(kind = 'bar', layout = (2, 3), subplots = True, figsize = (12, 12)\
            , xlabel = 'Year', ylabel = 'MCHF', title = "Balance sheet of {}".format(symbol), \
            sharey = False)
# an event handler to deal with the drop list changes
def dropdown_symbol_eventhandler(change):
    company_symb = change.new
    for c in companylist:
        if c.symbol == change.new:
            if(isinstance(c.balance_sheet,  pd.DataFrame)):
                display_by_company(c.balance_sheet, c.symbol)
            else:
                print("No Available Data for {}".format(c.symbol))
            
        
        
droplist = widgets.Dropdown(
    options = available_stocks_list, 
    description = 'Compagnies symbol:'
)
droplist.observe(dropdown_symbol_eventhandler,  names = 'value')
droplist

Widget Javascript not detected.  It may not be installed or enabled properly.


,total assets,total liabilities,current assets,current liabilities,inventory
2016-12-31,121.216,25.200,19.919,24.421,18.311
2017-12-31,145.929,37.348,20.731,37.140,19.328
2018-12-31,169.429,60.323,20.930,60.323,38.888
2019-12-31,194.653,89.821,40.502,79.388,48.771
2020-12-31,276.084,118.838,41.770,111.350,53.392


,total assets,total liabilities,current assets,current liabilities,inventory,goodwill
2016-12-31,540.626,66.652,138.931,60.259,<NA>,<NA>
2017-12-31,490.753,107.299,221.170,71.014,0.757,44.406
2018-12-31,719.558,110.650,199.011,74.725,7.065,44.406
2019-12-31,1135.496,481.732,259.226,103.300,11.546,44.406
2020-12-31,1759.555,605.180,310.797,189.609,13.048,44.406


In [44]:
def display_by_company2(df, symbol):
    df = df[::-1]
    #ase = plt.subplots_adjust(left = 0.08,  bottom = 0.18,  right = 0.9,  top = 0.88,  wspace = 0.4,  hspace = 0.68)
    df.plot(kind = 'bar', layout = (2, 3), subplots = True, figsize = (12, 12)\
            , xlabel = 'Year', ylabel = 'MCHF', title = "Cash Flow of {}".format(symbol),\
            sharey = False)
    
def dropdown_symbol_eventhandler2(change):
    company_symb = change.new
    for c in companylist:
        if c.symbol == change.new:
            if(isinstance(c.cash_flow,  pd.DataFrame)):
                display_by_company2(c.cash_flow, c.symbol)
            else:
                print("No Available Data for {}".format(c.symbol))
            
        
        
droplist = widgets.Dropdown(
    options = available_stocks_list, 
    description = 'Compagnies symbol:'
)
droplist.observe(dropdown_symbol_eventhandler2,  names = 'value')
droplist
## only MED ,  LAND ,  ATRI and AMS have cash flow data

Widget Javascript not detected.  It may not be installed or enabled properly.


## In this section we will highlight the most interesting aspects of correlations between stocks prices and SPI (swiss performance index) for multiple sectors in Switzerland  

In [45]:
pearson_corr = pd.read_csv("clean_preprocess/data_warehouse/data_for_dashboard/pearson_matrix.csv", skip_blank_lines = True)
spearman_corr = pd.read_csv("clean_preprocess/data_warehouse/data_for_dashboard/spearman_matrix.csv", skip_blank_lines = True)
pearson_corr.set_index(pearson_corr.iloc[:,  0], inplace = True)
spearman_corr.set_index(spearman_corr.iloc[:,  0], inplace = True)
pearson_corr.drop(pearson_corr.columns[0], inplace = True, axis = 1)
spearman_corr.drop(spearman_corr.columns[0], inplace = True, axis = 1)
def highlight_number(row):
    return [
        'background-color: green; color: white' if abs(cell) > 0.75
        else ''
        for cell in row
    ]
spearman_corr.style.apply(highlight_number)\
  .format('{0:,.3f}') \
  .set_caption('Spearman correlation ratio between SPI index and Stocks') \
  .set_properties(padding = "20px",  border = '2px solid white')


,SPI Swiss Performance Index - Index total (including dividend reinvestment),SPI Swiss Performance Index - Banks,SPI Swiss Performance Index - Financial services,SPI Swiss Performance Index - Insurance,SPI Swiss Performance Index - Food and beverages,SPI Swiss Performance Index - Health care,SPI Swiss Performance Index - Registered shares,SPI Swiss Performance Index - Bearer shares and participation certificates,SMI Swiss Market Index (excluding dividend reinvestment)
Unnamed: 0,,,,,,,,,
ALC,-0.372,0.405,-0.378,0.088,-0.482,-0.516,-0.291,-0.605,-0.286
SIGN,0.555,-0.120,0.586,0.558,0.455,0.556,0.568,0.507,0.517
CEVA,-0.791,0.509,-0.786,-0.745,-0.866,-0.760,-0.783,-0.789,-0.646
MED,-0.819,0.707,-0.769,-0.587,-0.914,-0.843,-0.787,-0.883,-0.634
SENS,0.478,-0.215,0.460,0.533,0.481,0.513,0.466,0.499,0.404
LAND,-0.215,-0.407,-0.216,-0.291,-0.177,-0.155,-0.230,-0.162,-0.143
GALE,0.692,-0.059,0.654,0.674,0.674,0.751,0.683,0.721,0.704
PLAN,-0.681,0.371,-0.681,-0.437,-0.520,-0.543,-0.685,-0.493,-0.628
BRG,0.610,-0.149,0.640,0.573,0.593,0.597,0.600,0.596,0.465


In [46]:
pearson_corr.style.apply(highlight_number) \
  .format('{0:,.3f}') \
  .set_caption('Pearson correlation ratio between SPI index and Stocks') \
  .set_properties(padding = "20px",  border = '2px solid white')

,SPI Swiss Performance Index - Index total (including dividend reinvestment),SPI Swiss Performance Index - Banks,SPI Swiss Performance Index - Financial services,SPI Swiss Performance Index - Insurance,SPI Swiss Performance Index - Food and beverages,SPI Swiss Performance Index - Health care,SPI Swiss Performance Index - Registered shares,SPI Swiss Performance Index - Bearer shares and participation certificates,SMI Swiss Market Index (excluding dividend reinvestment)
Unnamed: 0,,,,,,,,,
ALC,-0.370,0.483,-0.364,0.135,-0.542,-0.620,-0.244,-0.729,-0.249
SIGN,0.296,-0.120,0.311,0.318,0.301,0.311,0.295,0.293,0.227
CEVA,-0.713,0.557,-0.681,-0.645,-0.759,-0.703,-0.695,-0.743,-0.574
MED,-0.550,0.205,-0.483,-0.258,-0.579,-0.532,-0.528,-0.594,-0.419
SENS,0.465,-0.196,0.454,0.454,0.449,0.481,0.460,0.442,0.395
LAND,-0.164,-0.381,-0.124,-0.260,-0.064,-0.106,-0.186,-0.039,-0.117
GALE,0.722,-0.158,0.710,0.653,0.742,0.758,0.710,0.755,0.690
PLAN,-0.737,0.260,-0.755,-0.468,-0.604,-0.607,-0.746,-0.543,-0.669
BRG,0.578,-0.144,0.617,0.533,0.547,0.533,0.576,0.547,0.443


# Now that we finished giving an insight about the companies and their financial insights and we had an insight of some correlations.
# We will use the cleaned and labeled data in the Data_for_ML folder which is a matrix of (observations, features) and a label vector (1/-1) 1 being in an uptrend situation and -1 being an downtrend situation

In [47]:
company_models = {}


class Company_ML:
    """
    This class will be a wrapper for ML models of each company
    """
    def __init__(self,  symbol):
        self.symbol = symbol
        #Import Feature tables for each stock 
        self.ml_data = pd.read_csv("clean_preprocess/data_warehouse/data_for_ML/{}_data.csv".format(self.symbol))
        self.ml_data = shuffle(self.ml_data)
        # Feature engineering : some features are not very relevant for learning
        self.ml_data = self.ml_data.drop({"4. close", "1. open", "3. low", "2. high"}, axis = 1)
        # Split data into train set and test set
        x_train, x_test = train_test_split(self.ml_data,  test_size = 0.2, random_state = 100)
        #seperate features from lables
        y_train = x_train.pop("Trend label")
        y_test = x_test.pop("Trend label")
        self.x_train=x_train
        self.x_test=x_test
        self.y_train=y_train
        self.y_test=y_test
        #will be used as ground truth for model evaluation
        self.ground_truth=y_test
        #MLP classifier with Logistic function as activation function and stochastic gradien descent as optimizer
        #The model is fitted on the spot the same applies for all models
        self.mlp_model = MLPClassifier(hidden_layer_sizes=(2,10,2),learning_rate = 'adaptive'\
                    , activation = 'tanh',  solver = 'adam',  max_iter = 400)\
                    .fit(x_train, y_train)
        self.rf_model = RandomForestClassifier(criterion = "gini",\
                            n_estimators = 200, random_state = 0, \
                             max_features = 6)\
                    .fit(x_train, y_train)
        self.dt_model = DecisionTreeClassifier(random_state = 0, max_features = 8, max_depth = 8).fit(x_train, y_train)
        self.bn_model = GaussianNB().fit(x_train, y_train)

        

## This is the last part of the Project the algorithm will trade using Alpaca API 
>>### The strategy of the trading is pretty simple, the MLP model will predict the next week's trend and we will use that to buy or short sell a given stock
>>### We will take profit as 60% of Bollinger bands upper expected values and we will stop loss at 40% from the Bollinger bands lower expected value.

In [48]:
## We will set API keys and base URL, those are provided by the plateform
api_key = 'PKANCFL5W0F8PIX95MSZ'
api_secret = 'Xm2onggxEwSwuw5tmKNqP7aGmdRfC8FUe3N8MjVJ'
base_url = 'https://paper-api.alpaca.markets'

# instantiate REST API
api = tradeapi.REST(api_key,  api_secret,  base_url,  api_version = 'v2')
#make sure to trade available symboles
for symbol in available_stocks_list:
    try:
        api.get_asset(symbol)
    except APIError as e:
        if(e.status_code !=  200):
            available_stocks_list.remove(symbol)
        else:
            pass
        
def get_up_low_BB(symbol):
    """
    this method will get lower and upper bollinger bands values for a given symbol
    """
    data = requests.get(BASE_URL+"""&interval=weekly&time_period=5&series_type=close\
    &symbol={}&function=BBANDS""".format(symbol))    
    data = data.json()
    data = data.pop('Technical Analysis: BBANDS')
    for value in data:
        l, u = data[value]['Real Lower Band'], data[value]['Real Upper Band']
        break
    return (float(l), float(u))

In [49]:
portfolio=100000

In [50]:

nb_api_call = 0
#time.sleep(60)
for symbol in available_stocks_list:
    #get latest price
    s = (api.get_barset(symbol, timeframe = 'minute', limit = 1))[symbol][0].h
    #based on the price we will calculate the quantity traded qt
    #we will allocate portfolio/number of stocks for each stock
    qt = math.floor(portfolio/((len(available_stocks_list))*s))
    #use the Random Forest Classifier to predict up or down trend
    prediction = Company_ML(symbol).rf_model.predict(\
                                          pd.read_csv("clean_preprocess/data_warehouse/data_for_ML/{}_data.csv"\
                                            .format(symbol)).\
                                          drop({"4. close", "1. open", "3. low", "2. high", "Trend label"}, \
                                               axis = 1)\
                                                .head(1))
    #pick position for the submiting the order
    position = "buy" if prediction == 1 else "sell"
    #get lower and upper bollinger bands values to ensure a take profit/stop loss strategy
    if(nb_api_call == 5):
        time.sleep(60)
        nb_api_call = 0
    # get bollinger bands values for each symbol
    l, u = get_up_low_BB(symbol)
    nb_api_call = nb_api_call+1
    #set loss and profit as dict for the submit_order function.
    #the stop orders will be 60% of the difference price estimated from upper band and 40% from lower band
    #Example if the actual price is 500$ and the bb shows a standard deviation of 100$ ( 600 estimated maximum price)
    #we will liquidate the order at 560$ the same applies to lower band
    loss=dict(limit_price=str(l-(0.4*(s-l))),stop_price=str(l-(0.4*(s-l))))
    profit=dict(limit_price=str((0.6*(u-s))+s),stop_price=str((0.6*(u-s))+s))
    try:
        api.submit_order(symbol = symbol, qty = qt, side = position,\
                       time_in_force = 'gtc',  type = 'market', \
                        take_profit = profit, stop_loss = loss)
    except APIError as e: 
        print(e)




/home/shedy/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


## The last part of this project is to see the accuracy of the machine learning model using Confusion matrix 

In [25]:
def confusion_matrix_eventhandler(change):
    company_symb = change.new
    model = Company_ML(company_symb)
    models = [model.mlp_model,model.dt_model,model.rf_model,model.bn_model]
    labels = model.ml_data.pop("Trend label")
    data = model.ml_data
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15,10),sharex=True,sharey=True,squeeze=False)
    for cls, ax in zip(models, axes.flatten()):
        plot_confusion_matrix(cls, 
                          data, 
                          labels, 
                          ax=ax, 
                          cmap='Blues')
        ax.title.set_text(type(cls).__name__)
    plt.show()
            
        
        
droplist = widgets.Dropdown(
    options = available_stocks_list, 
    description = 'Compagnies symbol:'
)
droplist.observe(confusion_matrix_eventhandler,  names = 'value')
droplist

Widget Javascript not detected.  It may not be installed or enabled properly.


# To understand more the behaviour of the classification we are going to plot the ROC curve showcasing if the model can decide quiet well for both classes and to understand more about the precision and the recall of the model

In [26]:
def roc_auc_eventhandler(change):
    company_symb = change.new
    model = Company_ML(company_symb)
    metrics.plot_roc_curve(model.rf_model, model.x_test, model.y_test)  
    plt.show() 
    
droplist = widgets.Dropdown(
    options = available_stocks_list, 
    description = 'Compagnies symbol:'
)
droplist.observe(roc_auc_eventhandler,  names = 'value')
droplist

Widget Javascript not detected.  It may not be installed or enabled properly.
